Below there is a data set created with students who wish to join the app 

In [ ]:
import sqlite3

def create_connection(db_file):
    """ Create a database connection to the SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)
    return conn

def create_table(conn):
    """ Create a table for the students """
    try:
        sql_create_students_table = """ CREATE TABLE IF NOT EXISTS students (
                                            id integer PRIMARY KEY,
                                            name text NOT NULL,
                                            email text NOT NULL,
                                            degree text
                                        ); """
        cursor = conn.cursor()
        cursor.execute(sql_create_students_table)
    except Exception as e:
        print(e)

def insert_student(conn, student):
    """ Insert a new student into the students table """
    sql = ''' INSERT INTO students(name,email,degree)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, student)
    conn.commit()
    return cur.lastrowid

def main():
    database = "uniE_share.db"

    # Create a database connection
    conn = create_connection(database)

    # Create students table
    if conn is not None:
        create_table(conn)

        # Sample student data
        students = [
            ("Alice Smith", "alice.ieu2023@student.ie.edu", "BBA BDA"),
            ("Bob Johnson", "bob.ieu2022@student.ie.edu", "BBA BDA"),
            ("Charlie Brown", "charlie.ieu2021@student.ie.edu", "Engineering"),
            ("Daisy Ridley", "daisy.ieu2024@student.ie.edu", "BBA BDA"),
            ("Evan Peters", "evan.peters@student.ie.edu", "BBA BDA"),
            ("Fiona Gallagher", "fiona.2025@student.ie.edu", "BBA BDA"),
            ("George Lucas", "george.arts2023@student.ie.edu", "Arts")
        ]

        # Insert students
        for student in students:
            insert_student(conn, student)

        print("Students data inserted successfully.")
    else:
        print("Error! cannot create the database connection.")

    conn.close()

if __name__ == '__main__':
    main()


Run the Script: This script can be executed in a Python environment. It will create a SQLite database named uniE_share.db and populate it with the sample student data.
Database Structure: The database contains a table students with columns for id, name, email, and degree.
Sample Data: The script includes a mix of students, some meeting the criteria (email format and degree) and some not.

Code for the user interface

In [4]:
import sqlite3
import re
import hashlib

def create_connection(db_file):
    """ Create a database connection to the SQLite database. """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)
    return conn

def validate_email(email):
    """
    Validate the email format.
    Format: name.ieu20XX@student.ie.edu
    """
    pattern = r"[a-zA-Z]+\.ieu20\d{2}@student\.ie\.edu"
    return re.match(pattern, email) is not None

def validate_degree(degree):
    """
    Validate the degree.
    Required degree: BBA BDA
    """
    return degree == "BBA BDA"

def hash_password(password):
    """
    Hash the password using SHA-256.
    """
    return hashlib.sha256(password.encode()).hexdigest()

def find_user_by_email_and_degree(conn, email, degree):
    """
    Find a user by email and degree.
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM students WHERE email=? AND degree=?", (email, degree))
    return cur.fetchone()

def get_password_input():
    """
    Prompt user for password input and confirmation.
    """
    while True:
        password = input("Create your password: ")
        confirm_password = input("Confirm your password: ")
        if password == confirm_password:
            return password
        else:
            print("Passwords do not match. Please try again.")

def main():
    database = "uniE_share.db"

    # Create a database connection
    conn = create_connection(database)

    # Hash table (dictionary) to store user data
    user_data = {}

    if conn is not None:
        print("Welcome to UniE-Share Login")

        while True:
            email = input("Enter your email (name.ieu20XX@student.ie.edu) or 'exit' to quit: ")
            if email.lower() == 'exit':
                print("Exiting the application.")
                break

            if validate_email(email):
                degree = input("Enter your degree (BBA BDA): ")
                if validate_degree(degree):
                    user = find_user_by_email_and_degree(conn, email, degree)
                    if user:
                        print("User already exists. Please log in or reset your password.")
                        break
                    else:
                        # Password creation and confirmation
                        password = get_password_input()
                        hashed_password = hash_password(password)

                        # Store the user information in the hash table
                        user_data[email] = {'degree': degree, 'password': hashed_password}
                        print("Account created successfully!")
                        break
                else:
                    print("Invalid degree. Please enter 'BBA BDA' or type 'exit' to quit.")
            else:
                print("Invalid email format. Please try again or type 'exit' to quit.")

        conn.close()
    else:
        print("Error! cannot create the database connection.")

    # For demonstration, print the hash table
    print("User data stored:", user_data)

if __name__ == '__main__':
    main()


Welcome to UniE-Share Login
Account created successfully!
User data stored: {'aolaechea.ieu2022@student.ie.edu': {'degree': 'BBA BDA', 'password': '20d2995509ff4357692e8c421f3f81b876b0b2391dc84108fd34d86edc33201a'}}


Password Creation: Users are prompted to create a password and confirm it.
Password Confirmation: The script ensures the confirmed password matches the initial one.
Hash Table Storage: User data, including hashed passwords, is stored in a dictionary (a form of hash table).

Below there is a code for the search page 

In [6]:
class Subject:
    def __init__(self, name, resources):
        self.name = name
        self.resources = resources

    def display_resources(self):
        print(f"Resources for {self.name}:")
        for resource in self.resources:
            print(resource)

def create_sorted_subjects():
    subjects = [
        Subject("Algorithms and Data Structures", ["Resource 9", "Resource 10"]),
        Subject("Building Powerful Relationships", ["Resource 7", "Resource 8"]),
        Subject("Mathematics for Data Management and Analysis", ["Resource 3", "Resource 4"]),
        Subject("Probability and Statistics", ["Resource 5", "Resource 6"]),
        Subject("Programming for Data Management and Analysis", ["Resource 1", "Resource 2"]),
        Subject("Technology", ["Resource 11", "Resource 12"]),
    ]
    return subjects

def binary_search_subjects(subjects, query):
    low = 0
    high = len(subjects) - 1

    while low <= high:
        mid = (low + high) // 2
        if subjects[mid].name.lower() == query.lower():
            return subjects[mid]
        elif subjects[mid].name.lower() < query.lower():
            low = mid + 1
        else:
            high = mid - 1

    return None

def main():
    subjects = create_sorted_subjects()

    while True:
        query = input("Enter a subject to search, or 'exit' to quit: ").lower()
        if query == 'exit':
            print("Exiting the application.")
            break

        subject = binary_search_subjects(subjects, query)
        if subject:
            subject.display_resources()
            print("Redirecting to the home page...")
            # In a real app, here you would redirect to the home page
            break
        else:
            print("Subject not found. Please try again.")

if __name__ == '__main__':
    main()


Resources for Algorithms and Data Structures:
Resource 9
Resource 10
Redirecting to the home page...


Subject Class: Represents each subject with a name and a list of resources.
Create Subjects: Initializes available subjects. This can be expanded or modified as needed.
Search Function: Allows users to search for subjects. It uses a dictionary (hash table) for efficient lookup.
User Interaction: Users can search for subjects. If the subject is found, its resources are displayed, and the user is prompted to the home page.
The subjects are stored in a sorted list.
The binary_search_subjects function implements binary search to find a subject based on the user's query.
The search is case-insensitive for user convenience.

Below there is the code for once the search has been done 

In [1]:
class Material:
    def __init__(self, name):
        self.name = name

class Subject:
    def __init__(self, name):
        self.name = name
        self.textbooks = [Material("Textbook 1"), Material("Textbook 2"), Material("Textbook 3")]
        self.tests = [Material("Test 1"), Material("Test 2"), Material("Test 3")]
        self.exercises = [Material("Exercise 1"), Material("Exercise 2"), Material("Exercise 3")]
        self.notes = [Material("Note 1"), Material("Note 2"), Material("Note 3")]

    def add_material(self, material, material_type):
        getattr(self, material_type).append(material)

    def display_materials(self, material_type):
        materials = getattr(self, material_type)
        for material in materials:
            print(material.name)
        return materials

    def find_material(self, material_type, material_name):
        materials = getattr(self, material_type)
        return next((m for m in materials if m.name.lower() == material_name.lower()), None)

class App:
    def __init__(self):
        self.subjects = self._create_subjects()

    def _create_subjects(self):
        subjects = {
            "programming": Subject("Programming for Data Management and Analysis"),
            "mathematics": Subject("Mathematics for Data Management and Analysis"),
            "statistics": Subject("Probability and Statistics"),
            "relationships": Subject("Building Powerful Relationships"),
            "algorithms": Subject("Algorithms and Data Structures"),
            "technology": Subject("Technology")
        }
        return subjects
    def run(self):
        while True:
            subject_name = self._get_input("\nEnter a subject to search (e.g., 'programming'), or type 'exit' to quit: ").lower()
            if subject_name == 'exit':
                print("Exiting the application.")
                break

            subject = self.subjects.get(subject_name)
            if subject:
                self._handle_subject_interaction(subject)
            else:
                print("Subject not found. Please try again.")

    def _get_input(self, prompt):
        user_input = input(prompt)
        if user_input.lower() == 'exit':
            print("Exiting the application.")
            exit(0)  # Exit the program immediately
        return user_input


    def _handle_subject_interaction(self, subject):
        material_types = ['textbooks', 'tests', 'exercises', 'notes']
        while True:
            print(f"\nOptions for {subject.name}:")
            print("Type 'Textbooks', 'Tests', 'Exercises', 'Notes' to view materials")
            print("Type 'Post' to add new material")
            print("Type 'Exit' to leave")

            choice = input("Enter your choice (e.g., 'Textbooks'): ").lower()

            if choice == 'exit':
                break
            elif choice == 'post':
                self._post_material(subject)
            elif choice in material_types:
                self._view_and_select_material(subject, choice)
            else:
                print("Invalid choice. Please try again.")
    def _view_and_select_material(self, subject, material_type):
        print(f"Available {material_type.capitalize()}:")
        materials = subject.display_materials(material_type)
        if not materials:
            print("No materials available.")
            return

        choice = input(f"Select a material to view (e.g., 'Textbook 2'), or type 'exit' to leave: ").lower()
        if choice == 'exit':
            return

        selected_material = subject.find_material(material_type, choice)
        if selected_material:
            print(f"You have selected: {selected_material.name}")
            if input("Would you like to post new material? (yes/no): ").lower() == 'yes':
                self._post_material(subject)
            else:
                print("Thank you!")  # Print thank you message
                exit(0)  # Exit the program immediately

    def _post_material(self, subject):
        material_type = input("Enter the type of material to post (e.g., 'textbook'): ").lower() + 's'
        material_name = input("Enter the name of the new material (e.g., 'Textbook 4'): ")
        subject.add_material(Material(material_name), material_type)
        print(f"{material_name} added to {subject.name}.")
        print("Thank you for your input!")  # Thank the user for their input and exit
        return  # Exit the method to end the interaction
    

# Running the app
if __name__ == '__main__':
    app = App()
    app.run()



Options for Programming for Data Management and Analysis:
Type 'Textbooks', 'Tests', 'Exercises', 'Notes' to view materials
Type 'Post' to add new material
Type 'Exit' to leave
Available Textbooks:
Textbook 1
Textbook 2
Textbook 3
You have selected: Textbook 2
Thank you!

Options for Programming for Data Management and Analysis:
Type 'Textbooks', 'Tests', 'Exercises', 'Notes' to view materials
Type 'Post' to add new material
Type 'Exit' to leave
Exiting the application.
Exiting the application.


Material Class: Represents individual materials (textbook, test, exercise, note).
Subject Class: Represents a subject, with methods to add and display materials.
App Class: The main application class that orchestrates the user interaction and manages subjects.
User Interaction: The user can search for subjects, view materials, and post new materials.

the user can press exit at any time in order to stop the program